In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import copy
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('../induction_motor_anomaly_detection/')
import modules,scaler


In [23]:
normal_data = pd.read_csv('../data/combined_data.csv')
anomalous_data = pd.read_csv('../anomalous_data/a_data2.csv')

In [24]:
normal_data.shape, anomalous_data.shape

((3170000, 3), (10000, 3))

In [29]:

train_data_features = modules.ElectricalFeatureExtractor(current_data=normal_data)
test_data_features = modules.ElectricalFeatureExtractor(current_data=anomalous_data)


In [30]:
train_scaler = scaler.Scaler()
train_scaler.fit_unlabelled_data(normal_data)


using MinMaxScaler()


In [31]:
train_scaled = train_scaler.transform(normal_data)
anomalous_scaled = train_scaler.transform(anomalous_data)

In [35]:
train_scaled.shape

(3170000, 3)

In [32]:
class LSTMAutoEncoder(Model):
    def __init__(self, activation='relu'):
        super(LSTMAutoEncoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.LSTM(64, activation=activation, return_sequences=True),
            tf.keras.layers.LSTM(32, activation=activation, return_sequences=True),
            tf.keras.layers.LSTM(16, activation=activation),
            tf.keras.layers.RepeatVector(3)])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.LSTM(16, activation=activation, return_sequences=True),
            tf.keras.layers.LSTM(32, activation=activation, return_sequences=True),
            tf.keras.layers.LSTM(64, activation=activation, return_sequences=True),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation="sigmoid"))])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [42]:
model = LSTMAutoEncoder()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=2,
                                                  mode='min')
model.compile(optimizer='adam', loss='mae')



In [ ]:
for i in range(0,len(train_scaled), 10000):
    print(i,i+10000)

In [ ]:
train_scaled_3d = train_scaled.reshape(train_scaled.shape[0], train_scaled.shape[1], 1)


In [45]:
file = 0
for i in range(0,len(train_scaled_3d), 10000):
  history = model.fit(train_scaled_3d[i:i + 10000], train_scaled_3d[i:i + 10000], #passing something as y, preferrably the same data, since this is an unsupervised algo.
            epochs=50, 
            batch_size=128,
            validation_data=(train_scaled_3d[i:i + 10000], train_scaled_3d[i:i + 10000]), #passing combined data, to make model train harder, as per author.
            shuffle=True,
          #  We can and probably should use normal data only, for val, since we are fitting the model to be compress and decompress normal data, as well as detect abnormal data if reconstruction error is high
            callbacks=[early_stopping], verbose = 0)

ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [36]:
train_scaled_3d = train_scaled.reshape(train_scaled.shape[0], train_scaled.shape[1], 1)
from tqdm import tqdm

file = 0
for i in tqdm(range(0, len(train_scaled_3d), 10000)):
    history = model.fit(train_scaled_3d[0:1000], train_scaled_3d[0:1000],
                        epochs=50,
                        batch_size=128,
                        validation_data=(train_scaled_3d[0:1000], train_scaled_3d[0:1000]),
                        shuffle=True,
                        callbacks=[early_stopping],
                        verbose=0)


  0%|          | 0/317 [00:08<?, ?it/s]


ValueError: Creating variables on a non-first call to a function decorated with tf.function.